In [2]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.


In [3]:
import os
import cv2
import pandas as pd
import mediapipe as mp
from sklearn.metrics import euclidean_distances
import matplotlib.pyplot as plt

In [9]:
def extract_body_proportions(image_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)

    image = cv2.imread(image_path)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        chest = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y +
                 landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y) / 2
        waist = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y +
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y) / 2
        hip = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y +
               landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y) / 2
        shoulder = abs(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x -
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)

        return chest, waist, hip, shoulder
    return None, None, None, None

def create_dataset(base_folder, gender, celebrity_heights, output_csv):
    gender_folder = os.path.join(base_folder, gender)
    data = []

    for celebrity in os.listdir(gender_folder):
        celebrity_folder = os.path.join(gender_folder, celebrity)

        if os.path.isdir(celebrity_folder):
            image_path = os.path.join(celebrity_folder, f'{celebrity}_0.jpg')

            if os.path.exists(image_path):
                height = celebrity_heights.get(celebrity, None)

                chest, waist, hip, shoulder = extract_body_proportions(image_path)

                if chest is not None and height is not None:
                    data.append({
                        'Celebrity': celebrity,
                        'Chest': chest * height,  # Scale based on height for reference
                        'Waist': waist * height,
                        'Hip': hip * height,
                        'Shoulder': shoulder * height
                    })


    df = pd.DataFrame(data, columns=['Celebrity', 'Chest', 'Waist', 'Hip', 'Shoulder'])
    df.to_csv(output_csv, index=False)

base_folder = '/content/drive/MyDrive/Thuli'
celebrity_heights = { #height in cm
    'samantha' : 158,
    'priyanka_chopra' : 167.6,
    'deepika_padukone' : 170,
    'vidya_balan' : 163,
    'ranveer_singh' : 173,
    'hrithik_roshan' : 180,
    'kamal_haasan' : 165,
    'suriya' : 165,
    'rajkumar_rao' : 170
}

output_csv_male = 'male_celebrity_body_proportions.csv'
create_dataset(base_folder, 'Male', celebrity_heights, output_csv_male)

output_csv_female = 'female_celebrity_body_proportions.csv'
create_dataset(base_folder, 'Female', celebrity_heights, output_csv_female)

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [10]:
male = pd.read_csv('/content/male_celebrity_body_proportions.csv')

male

,Celebrity,Chest,Waist,Hip,Shoulder
0,ranveer_singh,36.565941,82.818417,116.631305,44.368783
1,kamal_haasan,40.799436,85.854167,114.602812,51.132042
2,suriya,40.601043,84.358535,117.984722,50.933881


In [11]:
female = pd.read_csv('/content/female_celebrity_body_proportions.csv')

female

,Celebrity,Chest,Waist,Hip,Shoulder
0,vidya_balan,34.695389,75.120428,107.371183,41.032392
1,priyanka_chopra,52.572624,85.244986,112.408233,28.826559
2,samantha,35.270634,71.738512,99.824894,84.410416


In [8]:
'''def extract_body_proportions(image_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)

    image = cv2.imread(image_path)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        chest = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y +
                 landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y) / 2
        waist = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y +
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y) / 2
        hip = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y +
               landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y) / 2
        shoulder = abs(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x -
                       landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x)

        return chest, waist, hip, shoulder
    return None, None, None, None'''

def load_celebrity_data(csv_path):
    return pd.read_csv(csv_path)

def find_most_similar_celebrity(person_proportions, celebrity_df):  #find similar celebrity
    person_array = [person_proportions['Chest'], person_proportions['Waist'],
                    person_proportions['Hip'], person_proportions['Shoulder']]

    celebrity_array = celebrity_df[['Chest', 'Waist', 'Hip', 'Shoulder']].values
    distances = euclidean_distances([person_array], celebrity_array)[0]

    closest_idx = distances.argmin()
    return celebrity_df.iloc[closest_idx]['Celebrity']

'''def display_celebrity_images(base_folder, celebrity_name):
    celebrity_folder = os.path.join(base_folder, 'Male', celebrity_name)

    if os.path.exists(celebrity_folder):
        for image_file in os.listdir(celebrity_folder):
            if image_file != f'{celebrity_name}_0.jpg':
                image_path = os.path.join(celebrity_folder, image_file)
                image = cv2.imread(image_path)
                #cv2.imshow(f'{celebrity_name} - {image_file}', image)
                cv2.imshow(image)
                cv2.waitKey(0)  # Press any key to move to the next image
        cv2.destroyAllWindows()
'''

def process_and_compare(input_image_path, input_height, csv_path, base_folder):
    chest, waist, hip, shoulder = extract_body_proportions(input_image_path)
    if chest is None:
        print("Error: Unable to extract body proportions from the input image.")
        return

    person_proportions = {
        'Chest': chest * input_height,
        'Waist': waist * input_height,
        'Hip': hip * input_height,
        'Shoulder': shoulder * input_height
    }

    celebrity_df = load_celebrity_data(csv_path)

    similar_celebrity = find_most_similar_celebrity(person_proportions, celebrity_df)
    print(f'The most similar celebrity is: {similar_celebrity}')

    #display_celebrity_images(base_folder, similar_celebrity)

input_image_path = '/content/drive/MyDrive/Thuli/Male/vikram.png'
input_height = 168  # height in cm
csv_path = 'male_celebrity_body_proportions.csv'
base_folder = '/content/drive/MyDrive/Thuli'

process_and_compare(input_image_path, input_height, csv_path, base_folder)


The most similar celebrity is: suriya


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
